In [3]:
import importlib
import funciones_analisis.estadisticas

importlib.reload(funciones_analisis.estadisticas)
importlib.reload(funciones_analisis.generales)

from funciones_analisis.estadisticas import *
from funciones_analisis.generales import *
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch, Pitch
from highlight_text import ax_text, fig_text
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patheffects as path_effects
import seaborn as sns
import glob

In [13]:
free_comps = sb.competitions()

free_comps[free_comps["competition_name"] == "Women's World Cup"]


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
72,72,107,International,Women's World Cup,female,False,True,2023,2024-07-14T16:59:48.469596,2024-07-14T17:01:38.887279,2024-07-14T17:01:38.887279,2024-07-14T16:59:48.469596
73,72,30,International,Women's World Cup,female,False,True,2019,2024-02-13T09:41:26.379767,2021-06-13T16:17:31.694,None,2024-02-13T09:41:26.379767


In [5]:
# Extraer únicos con país y si es internacional
df_unique = free_comps[['competition_name', 'country_name']].drop_duplicates()

# Ordenar por nombre
df_unique = df_unique.sort_values('competition_name')

# Guardar resultado
df_unique.to_csv("datos/otros/competitions_with_country.csv", index=False)


In [14]:
!pip install pillow



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from PIL import Image
import os
import math

# Carpeta donde están los logos
carpeta = "imagenes/competiciones_statsbomb"

# Parámetros
max_ancho, max_alto = 200, 200  # Tamaño máximo para cada logo (sin deformar)
espaciado = 30                  # Espacio entre logos
columnas = 5                    # Número de columnas

# Cargar imágenes y escalarlas proporcionalmente
imagenes_escaladas = []
for archivo in os.listdir(carpeta):
    if archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
        ruta = os.path.join(carpeta, archivo)
        img = Image.open(ruta).convert("RGBA")

        # Escalado proporcional
        img.thumbnail((max_ancho, max_alto), Image.LANCZOS)
        imagenes_escaladas.append(img)

# Calcular tamaño del collage
num_imagenes = len(imagenes_escaladas)
filas = math.ceil(num_imagenes / columnas)
ancho_total = columnas * max_ancho + (columnas + 1) * espaciado
alto_total = filas * max_alto + (filas + 1) * espaciado

# Crear fondo blanco
collage = Image.new("RGBA", (ancho_total, alto_total), (255, 255, 255, 255))

# Pegar imágenes centradas en cada celda
x_inicial, y_inicial = espaciado, espaciado
for i, img in enumerate(imagenes_escaladas):
    fila = i // columnas
    columna = i % columnas

    x_celda = columna * (max_ancho + espaciado) + espaciado
    y_celda = fila * (max_alto + espaciado) + espaciado

    # Centrar la imagen en su celda
    x_offset = x_celda + (max_ancho - img.width) // 2
    y_offset = y_celda + (max_alto - img.height) // 2

    collage.paste(img, (x_offset, y_offset), img)

# Guardar el collage final
collage.save("imagenes/collage_competiciones.png")
print("✅ Collage generado como collage_competiciones.png (manteniendo proporciones)")


✅ Collage generado como collage_competiciones.png (manteniendo proporciones)
